<a href="https://colab.research.google.com/github/ecsquare/Covid-19/blob/master/DMMwithGibsSampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd /content/gdrive/My\ Drive/Colab\ Notebooks/vegan_tweets_2020-05-/

/content/gdrive/My Drive/Colab Notebooks/vegan_tweets_2020-05-


In [0]:
!ls

 online_btm.html	    vegan_2020-05-03.json   vegan_2020-05-06.json
 vegan_2020-05-01.json	    vegan_2020-05-04.json  'vegan_2020-05-07 2.json'
'vegan_2020-05-02 2.json'   vegan_2020-05-05.json


In [0]:
import json

data = [json.loads(line)['tweet'] for line in open('vegan_2020-05-01.json', 'r')]

In [0]:
import string
import re
def clean_tweets(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

In [0]:
data = [clean_tweets(d) for d in data]


In [0]:
cd /content/gdrive/My\ Drive/Colab\ Notebooks/

/content/gdrive/My Drive/Colab Notebooks


In [0]:
ls

DMMwithGibsSampling.ipynb  Short-text_based_topics.ipynb
GibbsSamplingDMM.py        vegan_tweets_2020-05-/
Objd/                      week3_task2_fine_tuning_clean.ipynb
object_detection


In [0]:
from GibbsSamplingDMM import GibbsSamplingDMM

model = dmm.GibbsSamplingDMM(check_arg(sys.argv[1:]))
model.analyseCorpus()
model.topicAssigmentInitialise()
model.inference()
	
print "Writing Results"
model.writeTopTopicalWords()
model.writeTopicAssignments()
